In [1]:
import numpy as np

In [2]:
def calculate_x_n(a, b, c, d, n):
    #alpha = [0.0] * n
    #beta = [0.0] * n
    alpha = np.zeros(n)
    beta = np.zeros(n)    
    alpha[0] = -c[0] / b[0]
    beta[0] = d[0] / b[0]
    for i in range(1,n-1):
        #print(i)
        alpha[i] = -c[i] / (a[i-1]*alpha[i-1] + b[i])
        beta[i] = (d[i] - a[i-1]*beta[i-1]) / (a[i-1]*alpha[i-1] + b[i])
    x_n = (d[n-1] - a[n-2]*beta[n-2]) / (a[n-2]*alpha[n-2] + b[n-1])
    #print(alpha, beta)
    return x_n, alpha, beta

def calculate_c_n_vector(x_n, alpha, beta, n):
    x = [0.0] * (n-1)
    x.append(x_n)
    for i in range(n-2, -1, -1):
        x[i] = alpha[i]*x[i+1] + beta[i]
    return x

def calculate_d(y_n, h, n):
    d = [y_n[0] / 3 * (h*h)]
    for i in range(2, n+1):
        d.append(y_n[i] - 2*y_n[i-1] + y_n[i-2])
    return (3/(h*h)) * np.asarray(d)


a = 0
b = 1
n = 9
a_coefs = [1.0] * (n-1)
b_coefs = [4.0] * n
c_coefs = [1.0] * (n-1)
x_n = np.arange(a, b + b/18, b/9)
y_n = np.exp(x_n)
print('X: ',x_n)
print('Y: ',y_n)

X:  [0.         0.11111111 0.22222222 0.33333333 0.44444444 0.55555556
 0.66666667 0.77777778 0.88888889 1.        ]
Y:  [1.         1.11751907 1.24884887 1.39561243 1.5596235  1.742909
 1.94773404 2.17662993 2.43242545 2.71828183]


In [32]:
h = (b - a) / n
d_coefs = calculate_d(y_n, h, n)
#print("---", d_coefs)
spl, alpha, beta = calculate_x_n(a_coefs, b_coefs, c_coefs, d_coefs, n)
#print(spl, alpha, beta)

--- [1.         3.35600776 3.75040267 4.19114649 4.68368613 5.23410856
 5.84921612 6.53661055 7.30478694]
1.5787579042933786 [-0.25       -0.26666667 -0.26785714 -0.26794258 -0.26794872 -0.26794916
 -0.26794919 -0.26794919  0.        ] [0.25       0.82826874 0.78271445 0.91326409 1.01027975 1.13177137
 1.2640355  1.41278223 0.        ]


In [4]:
c_n = calculate_c_n_vector(spl, alpha, beta, n)
c_n.append(0)
c_n = np.asarray(c_n)
print('c_coefs: ', c_n)
d_n = (c_n[1:n+1] - c_n[:n]) / (3*h)
print('d_coefs: ',d_n)
a_n = y_n[:n]
print('a_coefs: ',a_n)
b_n = (y_n[1:n+1] - y_n[:n]) / h - (h / 3) * (c_n[1:n+1] + 2*c_n[:n])
print('b_coefs: ',b_n)

c_coefs:  [0.0825315  0.669874   0.59398026 0.70460761 0.77873579 0.86413535
 0.99883136 0.98975532 1.5787579  0.        ]
d_coefs:  [ 1.76202749 -0.2276812   0.33188204  0.22238455  0.25619868  0.40408803
 -0.02722812  1.76700775 -4.73627371]
a_coefs:  [1.         1.11751907 1.24884887 1.39561243 1.5596235  1.742909
 1.94773404 2.17662993 2.43242545]
b_coefs:  [1.02674803 1.11034864 1.25077689 1.39506443 1.55988036 1.7424216
 1.9494179  2.17037198 2.45576234]


In [25]:
x_n_new = np.arange(a, b + b/9, b/18)
y_n_new = np.exp(x_n_new)
spline_extended = [a_n[(idx-1)//2] + b_n[(idx-1)//2] * (x_n_new[idx] -x_n[(idx-1)//2]) + c_n[(idx-1)//2] * (x_n_new[idx] - x_n[(idx-1)//2])**2 +\
                   d_n[(idx-1)//2] * (x_n_new[idx] - x_n[(idx-1)//2])**3 for idx in range(1, len(a_n)*2+1)]
spline_extended = np.concatenate((a_n[:1], spline_extended))

In [22]:
spline = a_n + b_n*(x_n[1:n+1] - x_n[:n]) + c_n[:n]*(x_n[1:n+1] - x_n[:n])**2 +\
         d_n*(x_n[1:n+1] - x_n[:n])**3
spline = np.concatenate((a_n[:1], spline))

array([1.        , 1.11751907, 1.24884887, 1.39561243, 1.5596235 ,
       1.742909  , 1.94773404, 2.17662993, 2.43242545, 2.71828183])

In [26]:
for i in range(0,n+1):
    print("x=", x_n[i], " | f(x)=", y_n[i], " | spl(x)=",spline[i], " | delta=", np.abs(y_n[i]-spline[i]))

x= 0.0  | f(x)= 1.0  | spl(x)= 1.0  | delta= 0.0
x= 0.1111111111111111  | f(x)= 1.1175190687418637  | spl(x)= 1.1175190687418637  | delta= 0.0
x= 0.2222222222222222  | f(x)= 1.2488488690016821  | spl(x)= 1.2488488690016821  | delta= 0.0
x= 0.3333333333333333  | f(x)= 1.3956124250860895  | spl(x)= 1.3956124250860895  | delta= 0.0
x= 0.4444444444444444  | f(x)= 1.5596234976067807  | spl(x)= 1.5596234976067807  | delta= 0.0
x= 0.5555555555555556  | f(x)= 1.7429089986334578  | spl(x)= 1.7429089986334578  | delta= 0.0
x= 0.6666666666666666  | f(x)= 1.9477340410546757  | spl(x)= 1.9477340410546755  | delta= 2.220446049250313e-16
x= 0.7777777777777777  | f(x)= 2.176629931716248  | spl(x)= 2.176629931716248  | delta= 0.0
x= 0.8888888888888888  | f(x)= 2.4324254542872077  | spl(x)= 2.4324254542872077  | delta= 0.0
x= 1.0  | f(x)= 2.718281828459045  | spl(x)= 2.718281828459045  | delta= 0.0


In [28]:
for i in range(0,2*n+1):
    print("x=", x_n_new[i], " | f(x)=", y_n_new[i], " | spl(x)=",spline_extended[i], " | delta=", np.abs(y_n_new[i]-spline_extended[i]))

x= 0.0  | f(x)= 1.0  | spl(x)= 1.0  | delta= 0.0
x= 0.05555555555555555  | f(x)= 1.0571277447602365  | spl(x)= 1.0575984147738515  | delta= 0.0004706700136150044
x= 0.1111111111111111  | f(x)= 1.1175190687418637  | spl(x)= 1.1175190687418637  | delta= 0.0
x= 0.16666666666666666  | f(x)= 1.1813604128656459  | spl(x)= 1.181233576490805  | delta= 0.00012683637484078858
x= 0.2222222222222222  | f(x)= 1.2488488690016821  | spl(x)= 1.2488488690016821  | delta= 0.0
x= 0.2777777777777778  | f(x)= 1.3201927884341202  | spl(x)= 1.3202266534113838  | delta= 3.386497726354243e-05
x= 0.3333333333333333  | f(x)= 1.3956124250860895  | spl(x)= 1.3956124250860895  | delta= 0.0
x= 0.38888888888888884  | f(x)= 1.4753406154906221  | spl(x)= 1.475328851159584  | delta= 1.1764331038222053e-05
x= 0.4444444444444444  | f(x)= 1.5596234976067807  | spl(x)= 1.5596234976067807  | delta= 0.0
x= 0.5  | f(x)= 1.6487212707001282  | spl(x)= 1.6487309531466745  | delta= 9.682446546310786e-06
x= 0.5555555555555556  | f(